In [1]:
# -*- coding: utf-8 -*-
# 选项 2: 图片转文本摘要，LLM 仅处理文本 RAG 实现示例
# 使用 MLLM (占位符) 生成图片摘要，使用文本嵌入模型索引原文+摘要，使用纯文本 LLM 生成

import sqlite3
import os
import numpy as np
from typing import List, Dict, Union, Optional, Tuple
import faiss
from transformers import AutoTokenizer, AutoModel # 用于文本嵌入
from PIL import Image
import torch
import zhipuai # 用于文本生成 LLM
import json
import time
import random

# --- 数据加载与图片关联函数 (与原示例相同) ---
def load_data_from_json_and_associate_images(json_path: str, image_dir: str) -> List[Dict]:
    """
    # 从 JSON 文件加载文档数据，并尝试在指定的图像目录中关联对应的图片文件。
    # (与原示例代码相同，此处省略详细注释)
    """
    if not os.path.exists(json_path):
        print(f"错误: 未找到 JSON 文件 '{json_path}'。")
        return []
    documents = []
    image_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff']
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
    except Exception as e:
        print(f"错误: 读取或解析 JSON 文件 '{json_path}' 失败: {e}")
        return []
    for item in json_data:
        doc_id = item.get('name')
        text_content = item.get('description')
        if not doc_id or not text_content:
            continue
        image_path = None
        if image_dir and os.path.exists(image_dir):
             for ext in image_extensions:
                 potential_image_path = os.path.join(image_dir, str(doc_id) + ext)
                 if os.path.exists(potential_image_path):
                     image_path = potential_image_path
                     break
        documents.append({
            'id': str(doc_id),
            'text': str(text_content) if text_content is not None else None,
            'image_path': image_path
        })
    print(f"已加载 {len(json_data)} 条记录，成功准备 {len(documents)} 个文档。")
    return documents

# --- 新增: 图片摘要生成器 (占位符) ---
class ImageSummarizer_Placeholder:
    """
    # 图片摘要生成器 (占位符):
    # 负责为给定的图像文件生成文本摘要。
    # !!! 注意: 这只是一个占位符实现 !!!
    # !!! 实际应用中需要调用真正的 MLLM (如 LLaVA, BLIP-2, GPT-4V API) 来生成摘要 !!!
    """
    def __init__(self, model_identifier="PlaceholderSummarizer"):
        print(f"ImageSummarizer (占位符) 初始化。模型标识: {model_identifier}")
        # 在实际实现中，这里会加载 MLLM 模型或初始化 API 客户端

    def summarize(self, image_path: str) -> Optional[str]:
        """生成图像摘要 (占位符实现)"""
        if not image_path or not os.path.exists(image_path):
            print(f"  - [摘要器占位符] 图像路径无效或文件不存在: {image_path}")
            return None
        try:
            # --- !!! 在此替换为调用 MLLM 生成摘要的逻辑 !!! ---
            # response = call_llava_or_gpt4v(image_path, prompt="详细描述这张电路图的关键元件和功能。")
            # summary = response.text

            # 占位符逻辑: 返回包含文件名的简单描述
            filename = os.path.basename(image_path)
            summary = f"[图像摘要占位符] 这是对图像文件 '{filename}' 的自动生成描述。实际摘要应包含电路类型、关键元件等信息。"
            print(f"  - [摘要器占位符] 为 '{filename}' 生成了摘要。")
            return summary
        except Exception as e:
            print(f"  - [摘要器占位符] 为图像 {image_path} 生成摘要时出错: {e}")
            return None

# --- 1. 文本编码器 (TextEncoder - 使用 SentenceTransformer 或类似模型) ---
class TextEncoder:
    """
    # 使用 Hugging Face Transformers 加载文本嵌入模型 (例如 Sentence-BERT, BGE)。
    # 负责将文本字符串转换为向量。
    """
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2", device: Optional[str] = None):
        # 常用模型:
        # sentence-transformers/all-MiniLM-L6-v2 (英文, 384维)
        # sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 (多语言, 384维)
        # BAAI/bge-large-en-v1.5 (英文, 1024维, 效果好)
        # BAAI/bge-large-zh-v1.5 (中文, 1024维)
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModel.from_pretrained(model_name)
            self.vector_dimension = self.model.config.hidden_size

            if device:
                self.device = torch.device(device)
            elif torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                self.device = torch.device("cpu")

            self.model.to(self.device)
            self.model.eval()
            print(f"TextEncoder 初始化成功，模型: {model_name}, 维度: {self.vector_dimension}, 设备: {self.device}")
        except Exception as e:
            print(f"加载文本嵌入模型 {model_name} 失败: {e}")
            raise e

    def encode(self, text: str) -> Optional[np.ndarray]:
        """编码单个文本字符串"""
        if not text: return None
        try:
            with torch.no_grad():
                # Tokenize input texts
                inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512).to(self.device)
                # Get hidden states
                outputs = self.model(**inputs)
                # Mean Pooling: Take the mean of the last hidden state across the sequence length dimension
                # Masking is important to ignore padding tokens
                attention_mask = inputs['attention_mask']
                last_hidden_state = outputs.last_hidden_state
                masked_hidden_state = last_hidden_state * attention_mask.unsqueeze(-1)
                sum_hidden_state = torch.sum(masked_hidden_state, dim=1)
                sum_mask = torch.sum(attention_mask, dim=1, keepdim=True)
                mean_pooled_vector = sum_hidden_state / torch.clamp(sum_mask, min=1e-9) # Avoid division by zero

                vector = mean_pooled_vector.squeeze().cpu().numpy().astype('float32')
                norm = np.linalg.norm(vector)
                return vector / norm if norm > 1e-6 else np.zeros_like(vector)
        except Exception as e:
            print(f"编码文本 '{text[:30]}...' 出错: {e}")
            return None

# --- 2. 索引器 (Indexer) - 选项 2 ---
# 为图像生成摘要，使用文本编码器索引原文+摘要，存储原文+摘要文本
class Indexer_Option2:
    """
    # 索引器 (选项2):
    # - 使用 ImageSummarizer 为图像生成文本摘要。
    # - 使用 TextEncoder 分别编码原始文本块和生成的图像摘要。
    # - 将所有这些 *文本向量* 存储到 Faiss 索引中。
    # - 将原始文本块和生成的 *图像摘要文本* 存储到 SQLite 数据库中。
    """
    def __init__(self, db_path: str, faiss_index_path: str, text_embed_model: str, image_summarizer: ImageSummarizer_Placeholder):
        self.db_path = db_path
        self.faiss_index_path = faiss_index_path
        self.encoder = TextEncoder(text_embed_model) # 使用纯文本编码器
        self.summarizer = image_summarizer # 图片摘要器
        self.vector_dimension = self.encoder.vector_dimension

        self._init_db()
        self._load_or_create_faiss_index()

    def _init_db(self):
        """初始化 SQLite 数据库和表 (只存储文本信息)"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                # vector_index_id: Faiss ID, 主键
                # doc_id: 原始文档 ID
                # content_source: 'original_text' 或 'image_summary'
                # text_content: 存储原始文本或图像摘要文本
                # original_image_path: 存储生成摘要的原始图片路径 (可选，用于追溯)
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS text_items (
                        vector_index_id INTEGER PRIMARY KEY,
                        doc_id TEXT NOT NULL,
                        content_source TEXT NOT NULL CHECK(content_source IN ('original_text', 'image_summary')),
                        text_content TEXT,
                        original_image_path TEXT
                    )
                ''')
                cursor.execute("CREATE INDEX IF NOT EXISTS idx_doc_id_source ON text_items (doc_id, content_source)")
                conn.commit()
                print(f"数据库已初始化或连接成功: {self.db_path}")
        except Exception as e: print(f"初始化数据库失败: {e}"); raise e

    def _load_or_create_faiss_index(self):
        """加载或创建 Faiss 向量索引 (只存文本向量)"""
        try:
            if os.path.exists(self.faiss_index_path):
                self.index = faiss.read_index(self.faiss_index_path)
                print(f"成功加载 Faiss 索引: {self.faiss_index_path}, 包含 {self.index.ntotal} 个向量。")
            else:
                print("未找到 Faiss 索引文件，创建新的空索引。")
                quantizer = faiss.IndexFlatIP(self.vector_dimension) # 内积
                self.index = faiss.IndexIDMap2(quantizer)
                print(f"创建了新的 Faiss IndexIDMap2 (内积) 索引 (维度: {self.vector_dimension})。")
        except Exception as e:
            print(f"加载或创建 Faiss 索引失败: {e}。将创建一个新的空索引。")
            quantizer = faiss.IndexFlatIP(self.vector_dimension)
            self.index = faiss.IndexIDMap2(quantizer)

    def index_documents(self, documents: List[Dict]):
        """
        # 对文档列表进行索引。
        # 先处理原始文本，再为图像生成摘要并处理摘要文本。
        """
        if not documents: return
        print(f"开始处理 {len(documents)} 个文档进行索引 (选项 2: 原文+图片摘要)...")

        vectors_to_add = []
        vector_ids_for_batch = []
        data_to_store = [] # (vec_id, doc_id, source, text, orig_img_path)

        start_vector_index_id = self.index.ntotal
        next_vector_index_id = start_vector_index_id

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        processed_count = 0
        skipped_duplicates = 0
        skipped_encoding_errors = 0
        summary_errors = 0

        for doc in documents:
            doc_id = doc.get('id')
            text = doc.get('text')
            image_path = doc.get('image_path')

            if not doc_id: continue

            # --- 1. 索引原始文本部分 ---
            if text:
                cursor.execute("SELECT 1 FROM text_items WHERE doc_id = ? AND content_source = 'original_text'", (doc_id,))
                if cursor.fetchone():
                    skipped_duplicates += 1
                else:
                    text_vector = self.encoder.encode(text)
                    if text_vector is not None:
                        vectors_to_add.append(text_vector)
                        vector_ids_for_batch.append(next_vector_index_id)
                        data_to_store.append((next_vector_index_id, doc_id, 'original_text', text, None))
                        next_vector_index_id += 1
                    else:
                        skipped_encoding_errors += 1

            # --- 2. 索引图像摘要部分 ---
            if image_path and os.path.exists(image_path):
                cursor.execute("SELECT 1 FROM text_items WHERE doc_id = ? AND content_source = 'image_summary'", (doc_id,))
                if cursor.fetchone():
                    skipped_duplicates += 1
                else:
                    # 生成摘要
                    summary_text = self.summarizer.summarize(image_path)
                    if summary_text:
                        # 编码摘要文本
                        summary_vector = self.encoder.encode(summary_text)
                        if summary_vector is not None:
                            vectors_to_add.append(summary_vector)
                            vector_ids_for_batch.append(next_vector_index_id)
                            # 存储摘要文本和原始图片路径
                            data_to_store.append((next_vector_index_id, doc_id, 'image_summary', summary_text, image_path))
                            next_vector_index_id += 1
                        else:
                            skipped_encoding_errors += 1
                    else:
                        summary_errors += 1
            elif image_path:
                # print(f"  警告: 图像文件不存在 {image_path}")
                pass

            processed_count += 1
            if processed_count % 50 == 0:
                 print(f"  已处理 {processed_count}/{len(documents)} 个原始文档...")

        # --- 批量添加到 Faiss 和 SQLite ---
        if vectors_to_add:
            try:
                vectors_np = np.array(vectors_to_add, dtype='float32')
                ids_np = np.array(vector_ids_for_batch, dtype='int64')
                self.index.add_with_ids(vectors_np, ids_np)
                print(f"成功向 Faiss 添加 {len(vectors_np)} 个新文本向量。当前总数: {self.index.ntotal}")

                cursor.executemany(
                    "INSERT INTO text_items (vector_index_id, doc_id, content_source, text_content, original_image_path) VALUES (?, ?, ?, ?, ?)",
                    data_to_store
                )
                conn.commit()
                print(f"成功向数据库存储 {len(data_to_store)} 条新文本记录。")

            except Exception as e:
                print(f"批量添加或存储时出错: {e}")
                conn.rollback()
        else:
            print("没有新的有效向量或数据需要添加。")

        conn.close()
        print(f"索引过程完成。总处理文档: {processed_count}, 跳过重复: {skipped_duplicates}, 跳过编码错误: {skipped_encoding_errors}, 图片摘要失败: {summary_errors}")


    def get_text_item_by_vector_index_id(self, vector_index_id: int) -> Optional[Dict]:
        """根据 vector_index_id 从数据库获取文本条目信息"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute(
                    "SELECT doc_id, content_source, text_content, original_image_path FROM text_items WHERE vector_index_id = ?",
                    (vector_index_id,)
                )
                row = cursor.fetchone()
                if row:
                    doc_id, source, text, img_path = row
                    return {
                        'vector_index_id': vector_index_id,
                        'doc_id': doc_id,
                        'source': source, # 'original_text' or 'image_summary'
                        'text': text, # 原始文本或摘要文本
                        'original_image_path': img_path # 如果是摘要，这里有原图路径
                    }
                return None
        except Exception as e:
             print(f"从数据库根据 vector_index_id {vector_index_id} 获取文本条目出错: {e}")
             return None

    def get_index_count(self) -> int: return self.index.ntotal if hasattr(self, 'index') else 0
    def get_db_item_count(self) -> int:
         try:
             with sqlite3.connect(self.db_path) as conn: cursor = conn.cursor(); cursor.execute("SELECT COUNT(*) FROM text_items"); return cursor.fetchone()[0]
         except Exception: return 0
    def save_index(self):
        if hasattr(self, 'index') and self.index.ntotal > 0:
            try: faiss.write_index(self.index, self.faiss_index_path); print(f"Faiss 索引已保存: {self.faiss_index_path}")
            except Exception as e: print(f"保存 Faiss 索引失败: {e}")
        elif hasattr(self, 'index'): print("索引为空，跳过保存。")
        else: print("索引未初始化，跳过保存。")
    def close(self): self.save_index()

# --- 3. 检索器 (Retriever) - 选项 2 ---
# 使用文本编码器对文本查询编码，在 Faiss 中搜索文本向量，获取原文或摘要文本
class Retriever_Option2:
    """
    # 检索器 (选项2):
    # - 使用 Indexer 的 TextEncoder 对用户的 *纯文本查询* 进行编码。
    # - 在 Faiss 索引（只包含文本向量）中搜索最相似的文本向量。
    # - 根据返回的 vector_index_id 从数据库获取对应的原始文本块或图像摘要文本。
    """
    def __init__(self, indexer: Indexer_Option2):
        if not isinstance(indexer, Indexer_Option2): raise ValueError("必须提供有效的 Indexer_Option2 实例。")
        self.indexer = indexer
        self.encoder = indexer.encoder # TextEncoder
        self.index = indexer.index
        if self.index is None or self.index.ntotal == 0: print("警告: 检索器初始化时发现索引为空。")

    def retrieve(self, query: str, k: int = 5) -> List[Dict]:
        """
        # 执行检索流程 (仅限文本查询)。

        Args:
            query: 用户纯文本查询。
            k: 检索数量。

        Returns:
            包含检索到的文本内容（原文或摘要）和得分的字典列表。
            [{'doc_id': ..., 'source': ..., 'text': ..., 'original_image_path': ..., 'score': ...}, ...]
        """
        if self.index is None or self.index.ntotal == 0: print("错误: 向量索引不可用或为空。"); return []
        if not isinstance(query, str) or not query: print("错误: 查询必须是非空文本字符串。"); return []

        # 1. 查询编码 (纯文本)
        query_vector = self.encoder.encode(query)
        if query_vector is None: print("错误: 查询编码失败。"); return []
        query_vector = query_vector.reshape(1, self.indexer.vector_dimension)

        # 2. Faiss 搜索 (文本向量)
        try:
            print(f"  - 正在 Faiss 索引中搜索 Top {k} 文本向量...")
            scores, vector_index_ids = self.index.search(query_vector, k)
            print(f"  - Faiss 搜索完成，找到 {len(vector_index_ids[0])} 个结果。")
        except Exception as e: print(f"  - Faiss 搜索失败: {e}"); return []

        # 3. 获取文本数据 (原文或摘要)
        retrieved_texts = []
        print("  - 正在从数据库获取文本条目信息...")
        for i, vec_id in enumerate(vector_index_ids[0]):
            if vec_id == -1: continue
            text_item_data = self.indexer.get_text_item_by_vector_index_id(int(vec_id))
            if text_item_data:
                text_item_data['score'] = float(scores[0][i])
                retrieved_texts.append(text_item_data)
            else: print(f"  - 警告: 未找到 vector_index_id {vec_id} 对应的数据库记录。")
        print(f"  - 成功获取 {len(retrieved_texts)} 个文本条目的数据。")
        return retrieved_texts

    def close(self): pass

# --- 4. 生成器 (Generator) - 选项 2 (使用纯文本 LLM) ---
# 与原示例的 Generator 基本相同，接收纯文本上下文
class Generator_Option2:
    """
    # 生成器 (选项2):
    # - 使用标准的纯文本大语言模型 (如 ZhipuAI GLM-4-flash)。
    # - 接收检索到的纯文本上下文（原文片段 + 图片摘要文本）。
    # - 构建适合文本 LLM 的 Prompt。
    # - 调用文本 LLM API 生成答案。
    """
    def __init__(self, api_key: Optional[str] = None, model_name: str = "glm-4-flash"):
        final_api_key = api_key if api_key else os.getenv("ZHIPUAI_API_KEY")
        if not final_api_key: raise ValueError("未提供 ZHIPUAI_API_KEY。")
        try:
            self.client = zhipuai.ZhipuAI(api_key=final_api_key)
            self.model_name = model_name
            print(f"Generator (选项 2 - 纯文本 LLM) 初始化成功。模型: {self.model_name}")
        except Exception as e: print(f"初始化智谱 AI 客户端失败: {e}"); raise e

    def generate(self, query: str, context: List[Dict]) -> str:
        """
        # 接收文本查询和纯文本上下文，生成答案。

        Args:
            query: 原始用户文本查询。
            context: 从 Retriever_Option2 获取的列表，每个元素包含文本内容。
                     [{'doc_id': ..., 'source': ..., 'text': ..., 'score': ...}, ...]

        Returns:
            LLM 生成的文本响应。
        """
        print(f"正在使用 {len(context)} 个检索到的文本条目为查询生成响应...")

        # 1. 构建 Prompt (与原示例类似，但上下文包含摘要)
        messages = self._build_messages(query, context)
        print("  - Prompt 构建完成。")

        # 2. 调用文本 LLM API
        print(f"  - 正在调用智谱 AI API (模型: {self.model_name})...")
        try:
            response = self.client.chat.completions.create(
                model=self.model_name, messages=messages, temperature=0.7, max_tokens=1024
            )
            llm_response = response.choices[0].message.content
            print("  - 智谱 AI API 调用成功。")
        except Exception as e:
            print(f"  - 调用 LLM 时出错: {e}")
            return f"调用 LLM 出错: {e}"

        # 3. 后处理
        processed_response = llm_response.strip()
        print("  - 响应生成和后处理完成。")
        return processed_response

    def _build_messages(self, query: str, context: List[Dict]) -> List[Dict]:
        """构建发送给纯文本 LLM 的消息列表"""
        system_message_content = """
        你是一个专业的文档问答助手。请严格根据以下提供的"参考信息"来回答用户的查询。
        # 重要规则:
        - **严格性:** 回答必须完全基于下面的 "参考信息"。不要使用外部知识。
        - **信息来源:** 参考信息可能来自原始文档文本，也可能来自对相关图片的文本描述(标记为 [图片摘要])。
        - **信息不足:** 如果信息不足以回答，请说明"根据提供的文档，我无法回答这个问题"。
        - **关于图片:** 你看到的是图片的文本描述，而不是图片本身。回答时请基于这些描述。

        # 参考信息:
        --- 开始参考信息 ---
        """.strip()

        context_text_parts = []
        if not context:
            context_text_parts.append("未找到相关信息。")
        else:
            for i, item in enumerate(context):
                doc_id = item.get('doc_id', 'N/A')
                score = item.get('score', 'N/A')
                text_content = item.get('text', '无内容')
                source_type = item.get('source', '未知来源')
                source_tag = "[图片摘要]" if source_type == 'image_summary' else "[原文文本]"

                # 格式化每个条目
                context_text_parts.append(f"信息 {i+1} (来源 Doc ID: {doc_id}, 类型: {source_tag}, 得分: {score:.4f}):")
                truncated_text = text_content[:500] + ('...' if len(text_content) > 500 else '')
                context_text_parts.append(truncated_text)
                context_text_parts.append("-" * 10)

            if context_text_parts and context_text_parts[-1] == "-" * 10: context_text_parts.pop()
            context_text_parts.append("--- 结束参考信息 ---")

        messages = [
            {"role": "system", "content": system_message_content + "\n" + "\n".join(context_text_parts)},
            {"role": "user", "content": query}
        ]
        return messages

# --- 选项 2 示例使用流程 ---
if __name__ == "__main__":
    print("\n" + "="*10 + " 选项 2: 图片转摘要+文本 RAG 示例 " + "="*10 + "\n")

    # --- 配置 ---
    json_data_path = 'data.json'
    image_directory_path = 'images'
    db_file_opt2 = 'rag_option2.db'
    faiss_index_file_opt2 = 'rag_option2.faiss'
    TEXT_EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2" # 选择文本嵌入模型
    TEXT_LLM_MODEL = "glm-4-flash" # 选择纯文本生成模型

    # --- 清理 ---
    print("--- 清理旧文件 ---")
    if os.path.exists(db_file_opt2): os.remove(db_file_opt2)
    if os.path.exists(faiss_index_file_opt2): os.remove(faiss_index_file_opt2)
    print("--- 清理完成 ---")

    # --- 1. 加载数据 ---
    print("\n--- 步骤 1: 加载数据 ---")
    documents = load_data_from_json_and_associate_images(json_path, image_directory_path)
    if not documents: exit("错误: 未加载到文档。")
    print("--- 步骤 1 完成 ---")

    # --- 1.5 初始化图片摘要器 (占位符) ---
    print("\n--- 步骤 1.5: 初始化图片摘要器 (占位符) ---")
    image_summarizer = ImageSummarizer_Placeholder()
    print("--- 步骤 1.5 完成 ---")

    # --- 2. 初始化 Indexer 并索引 ---
    print("\n--- 步骤 2: 初始化 Indexer (选项 2) 并索引 ---")
    indexer_opt2 = None
    try:
        indexer_opt2 = Indexer_Option2(
            db_path=db_file_opt2,
            faiss_index_path=faiss_index_file_opt2,
            text_embed_model=TEXT_EMBED_MODEL,
            image_summarizer=image_summarizer # 传入摘要器实例
        )
        indexer_opt2.index_documents(documents)
        print(f"索引完成。向量总数: {indexer_opt2.get_index_count()}, 数据库条目数: {indexer_opt2.get_db_item_count()}")
        if indexer_opt2.get_index_count() == 0: print("错误: 索引为空！"); indexer_opt2 = None
    except Exception as e:
        print(f"Indexer 初始化或索引失败: {e}")
        indexer_opt2 = None
    print("--- 步骤 2 完成 ---")

    # --- 3. 初始化 Retriever ---
    print("\n--- 步骤 3: 初始化 Retriever (选项 2) ---")
    retriever_opt2 = None
    if indexer_opt2:
        try:
            retriever_opt2 = Retriever_Option2(indexer=indexer_opt2)
            print("Retriever 初始化成功。")
        except Exception as e: print(f"Retriever 初始化失败: {e}")
    else: print("Indexer 不可用，跳过 Retriever 初始化。")
    print("--- 步骤 3 完成 ---")

    # --- 4. 初始化 Generator ---
    print("\n--- 步骤 4: 初始化 Generator (选项 2 - 纯文本 LLM) ---")
    generator_opt2 = None
    if os.getenv("ZHIPUAI_API_KEY"):
        try:
            generator_opt2 = Generator_Option2(model_name=TEXT_LLM_MODEL)
            print("Generator 初始化成功。")
        except Exception as e: print(f"Generator 初始化失败: {e}")
    else: print("ZHIPUAI_API_KEY 未设置，跳过 Generator 初始化。")
    print("--- 步骤 4 完成 ---")

    # --- 5. 执行查询 (仅文本) ---
    print("\n--- 步骤 5: 执行查询示例 (仅文本) ---")
    if retriever_opt2 and generator_opt2:
        # 辅助函数打印结果
        def print_retrieved_items_opt2(items: List[Dict]):
            if not items: print("    (未检索到条目)")
            for i, item in enumerate(items):
                score_str = f"得分: {item.get('score', 'N/A'):.4f}"
                source_tag = "[图片摘要]" if item['source'] == 'image_summary' else "[原文文本]"
                print(f"    {i+1}. {source_tag} ID: {item['doc_id']}, {score_str}")
                text_preview = item['text'][:150] + ('...' if len(item['text']) > 150 else '')
                print(f"       文本: {text_preview}")
                if item.get('original_image_path'):
                     print(f"       (来自图像: {os.path.basename(item['original_image_path'])})") # 显示来源图片
                print("    " + "-" * 15)

        # 查询示例 (纯文本)
        queries_opt2 = [
            "带隙基准电路的核心原理是什么？",
            "请根据图片摘要描述基于 LM317 的电路。", # 尝试通过摘要提问
            "CTAT 电压是如何补偿温度变化的？",
        ]

        for i, query in enumerate(queries_opt2):
            print(f"\n--- 文本查询 {i+1}/{len(queries_opt2)} ---")
            print(f"查询: {query}")

            try:
                retrieved_context = retriever_opt2.retrieve(query, k=5)
                print(f"  检索到 {len(retrieved_context)} 个文本条目:")
                print_retrieved_items_opt2(retrieved_context)

                if retrieved_context:
                    print("\n  生成响应 (使用纯文本 LLM):")
                    response = generator_opt2.generate(query, retrieved_context)
                    print(response)
                else:
                    print("\n  未检索到上下文，无法生成响应。")
            except Exception as e:
                print(f"\n  执行查询或生成时出错: {e}")
            time.sleep(1)

    else:
        print("Retriever 或 Generator 未初始化，跳过查询执行。")
    print("--- 步骤 5 完成 ---")

    # --- 6. 清理 ---
    print("\n--- 步骤 6: 清理资源 ---")
    if indexer_opt2: indexer_opt2.close()
    # Retriever 和 Generator 无需关闭
    print("--- 清理完成 ---")
    print("\n" + "="*10 + " 选项 2 示例结束 " + "="*10 + "\n")

d:\ProgramData\anaconda3\envs\mrag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



========== 选项 2: 图片转摘要+文本 RAG 示例 ==========

--- 清理旧文件 ---
--- 清理完成 ---

--- 步骤 1: 加载数据 ---


NameError: name 'json_path' is not defined